# Managing permissions

This example requires the Pro-feature to be active

In [1]:
import geoengine as ge

## Initialize Geo Engine as Admin

In [ ]:
ge.initialize("http://localhost:3030/api", ("admin@localhost", "adminadmin"))

## Create layer

In [3]:
root_collection = ge.layer_collection()
root_of_layerdb = root_collection.items[0].load()
layer_id = root_of_layerdb.add_layer(
    name="ports clone",
    description="test description",
    workflow={
        "type": "Vector",
        "operator": {
            "type": "PointInPolygonFilter",
            "params": {},
            "sources": {
                "points": {
                    "type": "OgrSource",
                    "params": {
                        "data": "ne_10m_ports",
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                },
                "polygons": {
                    "type": "OgrSource",
                    "params": {
                        "data": "germany_outline",
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                }
            }
        }
    },
    symbology=None,
)

root_of_layerdb.reload()
root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: e934ac59-9610-4056-ba21-424a44b15237
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Login anonymously, the layer is not visible

In [4]:
ge.initialize("http://localhost:3030/api")
root_collection = ge.layer_collection()
root_of_layerdb = root_collection.items[0].load()

root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Share layer with all anonymous users

In [5]:
ge.initialize("http://localhost:3030/api", ("admin@localhost", "adminadmin"))

resource = ge.Resource.from_layer_id(layer_id)
ge.add_permission(ge.ANONYMOUS_USER_ROLE_ID, resource, ge.Permission.READ)

## Layer is now visible for anonymous users

In [6]:
ge.initialize("http://localhost:3030/api")
root_collection = ge.layer_collection()
root_of_layerdb = root_collection.items[0].load()

root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: e934ac59-9610-4056-ba21-424a44b15237
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Create a user and a role

In [7]:
import requests as re
email = "foo@example.com"
password = "secret12345"

# register a user
response = re.post("http://localhost:3030/api/user", json={"email": email, "password": password, "realName": "Foo Bar"})
user_id = ge.UserId.from_response(response.json())

ge.initialize("http://localhost:3030/api", ("admin@localhost", "adminadmin"))

role_id = ge.add_role("Test Role")

{'id': 'e9e812b2-ae41-4976-8919-f2e9bf1383ce'}


## Create a layer and share it with new role

In [8]:
layer_id = root_of_layerdb.add_layer(
    name="ports clone for new role",
    description="test description",
    workflow={
        "type": "Vector",
        "operator": {
            "type": "PointInPolygonFilter",
            "params": {},
            "sources": {
                "points": {
                    "type": "OgrSource",
                    "params": {
                        "data": "ne_10m_ports",
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                },
                "polygons": {
                    "type": "OgrSource",
                    "params": {
                        "data": "germany_outline",
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                }
            }
        }
    },
    symbology=None,
)

resource = ge.Resource.from_layer_id(layer_id)
ge.add_permission(role_id, resource, ge.Permission.READ)

## Login as new user, the layer is not visible

In [9]:
ge.initialize("http://localhost:3030/api", (email, password))

root_collection = ge.layer_collection()
root_of_layerdb = root_collection.items[0].load()

root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Assign role to user, layer is now visible

In [10]:
ge.initialize("http://localhost:3030/api", ("admin@localhost", "adminadmin"))

ge.assign_role(role_id, user_id)

ge.initialize("http://localhost:3030/api", (email, password))

root_collection = ge.layer_collection()
root_of_layerdb = root_collection.items[0].load()

root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone for new role
description: test description
id: 7766a84c-964d-48c2-88d3-dbef82861c8a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74